# Snowflake Feature Store

Feature engineering, in which raw data is transformed into features that can be used to train machine learning models, is a vital part of building high-quality machine learning applications. A feature store lets you easily create, find, and employ features that work with your data.

The Snowflake Feature Store is designed to make creating, storing, and managing features for data science and machine learning workloads easier and more efficient. In this notebook, we will go through the steps to create and use Snowflake Feature Store.  

Adapted from https://quickstarts.snowflake.com/guide/intro-to-feature-store/index.html#2

## Prepare Snowpark Session

Create a session and set metadata

In [ ]:
from snowflake.ml.feature_store.examples.example_helper import ExampleHelper
from snowflake.snowpark.context import get_active_session

session = get_active_session()


# Add a query tag to the session. This helps with debugging and performance monitoring.
session.query_tag = {"origin":"sf_sit-is", "name":"aiml_notebooks_fs_overview", "version":{"major":1, "minor":0}, "attributes":{"is_quickstart":0, "source":"notebook"}}

# Set session context 
session.use_role("FEATURE_STORE_LAB_USER") 

# Print the current role, warehouse, and database/schema
print(f"role: {session.get_current_role()} | WH: {session.get_current_warehouse()} | DB.SCHEMA: {session.get_fully_qualified_current_schema()}")

In [ ]:
USE DATABASE FEATURE_STORE_DATABASE;
USE SCHEMA LIVE_DEMO_SCHEMA;

In [ ]:
# Print the current role, warehouse, and database/schema
print(f"role: {session.get_current_role()} | WH: {session.get_current_warehouse()} | DB.SCHEMA: {session.get_fully_qualified_current_schema()}")

## Prepare sample data

For this exercise, we will use the airline features data pulled from Snowflake-ml-python. See the [documentation]() for more information on the data.

In [ ]:
# If you loaded the tables directly, you can skip this step
example_helper = ExampleHelper(session, 
                               session.get_current_database(), 
                               session.get_current_schema())
source_tables = example_helper.load_example('airline_features')
source_tables

In [ ]:
us_flight_schedules = session.table("US_FLIGHT_SCHEDULES")
airport_weather_station = session.table("AIRPORT_WEATHER_STATION")

# Show a preview of the data using snowpark.DataFrame.to_pandas()
us_flight_schedules.limit(5).to_pandas()

In [ ]:
session.table("airport_weather_station").show()

Create a new Feature Store
=========================================

Create a new Feature Store from in our database and schema. Note that we also configure a
``default_warehouse`` to be used with the Feature Store. The choice of warehouse is not important at
this time so long as a valid warehouse is provided.

In [ ]:
from snowflake.ml.feature_store import FeatureStore, CreationMode, Entity, FeatureView

fs = FeatureStore(
    session=session,
    database="FEATURE_STORE_DATABASE",
    name="LIVE_DEMO_SCHEMA",
    default_warehouse="FEATURE_STORE_WH",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

## Creating Entities

An *entity* is an abstraction over a set of primary keys used for looking up feature data. An Entity represents a real-world "thing" that has data associated with it. The cell below registers two entities -- one for airports and the other airplanes.

In [5]:
zipcode_entity = Entity(
    name="AIRPORT_ZIP_CODE",
    join_keys=["AIRPORT_ZIP_CODE"],
    desc="Zip code of the airport.",
)
fs.register_entity(zipcode_entity)

plane_entity = Entity(name="PLANE_MODEL", join_keys=["PLANE_MODEL"], desc="The model of an airplane.")
fs.register_entity(plane_entity)

# Show our newly created entity
fs.list_entities().show()

--------------------------------------------------------------------------------------------------------------
|"NAME"  |"JOIN_KEYS"                          |"DESC"                                          |"OWNER"     |
--------------------------------------------------------------------------------------------------------------
|ROUTE   |["START_STATION_ID,END_STATION_ID"]  |Starting and ending stations for the bike ride  |REGTEST_RL  |
--------------------------------------------------------------------------------------------------------------



## Creating Feature Views

A *feature view* is a group of logically-related features that are refreshed on the same schedule. The
`FeatureView` constructor accepts a Snowpark DataFrame that contains the feature generation logic. The provided
DataFrame must contain the `join_keys` columns specified in the entities associated with the feature view. In
this example we are using time-series data, so we will also specify the timestamp column name. 

Below cell creates a feature view with 2 features. These 4 features are total rain over the past 30 and 60 minutes, grouped by entity (AIRPORT_ZIP_CODE).

In [6]:
airport_weather = session.sql(
    """
        SELECT
        datetime_utc AS ts,
        airport_zip_code,
        SUM(rain_mm_h) OVER (
            PARTITION BY airport_zip_code
            ORDER BY datetime_utc
            RANGE BETWEEN INTERVAL '30 minutes' PRECEDING AND CURRENT ROW
        ) AS rain_sum_30m,
        SUM(rain_mm_h) OVER (
            PARTITION BY airport_zip_code
            ORDER BY datetime_utc
            RANGE BETWEEN INTERVAL '60 minutes' PRECEDING AND CURRENT ROW
        ) AS rain_sum_60m
    FROM airport_weather_station
    """
)

airport_weather_fv = FeatureView(
    name="f_weather",  # name of feature view
    entities=[zipcode_entity],  # entities
    feature_df=airport_weather,  # definition query
    timestamp_col="TS",  # timestamp column
    refresh_freq="1d",  # refresh frequency
    desc="Airport weather features refreshed every day.",
).attach_feature_desc(
    {
        "RAIN_SUM_30M": "The sum of rain fall over past 30 minutes for one zipcode.",
        "RAIN_SUM_60M": "The sum of rain fall over past 1 hour for one zipcode.",
    }
)

airport_weather_fv = fs.register_feature_view(airport_weather_fv, version="1", overwrite=True)

# Show our newly created Feature View and display as Pandas DataFrame
fs.list_feature_views().to_pandas()

DataFrameAnalyticsFunctions.time_series_agg() is experimental since 1.12.0. Do not use it in production. 
DataFrame.alias() is experimental since 1.5.0. Do not use it in production. 


,NAME,VERSION,DATABASE_NAME,SCHEMA_NAME,CREATED_ON,OWNER,DESC,ENTITIES,REFRESH_FREQ,REFRESH_MODE,SCHEDULING_STATE,WAREHOUSE
0,TRIP_HISTORY,1,REGTEST_DB,SNOWFLAKE_FEATURE_STORE_NOTEBOOK_QUICK_START,2024-07-22 10:48:05.340,REGTEST_RL,,"[\n ""ROUTE""\n]",None,None,None,None


In [ ]:
plane = session.sql(
    """
    SELECT
        plane_model,
        seating_capacity,
        max_range_km
    FROM
        plane_model_attributes
    """
)

plane_fv = FeatureView(
    name="f_plane",  # name of feature view
    entities=[plane_entity],  # entities
    feature_df=plane,  # definition query
    refresh_freq=None,  # refresh frequency
    desc="Plane features never refresh.",
).attach_feature_desc(
    {
        "SEATING_CAPACITY": "The seating capacity of a plane.",
        "MAX_RANGE_KM": "The maximum range of a full tank.",
    }
)

plane_fv = fs.register_feature_view(plane_fv, version="1", overwrite=True)

# Show our newly created Feature View and display as Pandas DataFrame
fs.list_feature_views().to_pandas()

## Generating Datasets for Training

We are now ready to generate our training set. We'll define a spine DataFrame to form the backbone of our generated
dataset and pass it into ``FeatureStore.generate_dataset()`` along with our Feature Views.

> NOTE: The spine serves as a request template and specifies the entities, labels and timestamps (when applicable). The
  feature store then attaches feature values along the spine using an AS-OF join to efficiently combine and serve
  the relevant, point-in-time correct feature data.

In [ ]:
# Note that we use SCHEDULED_DEPARTURE_UTC as the timestamp in our spine, which will be matched to the timestamp column(s) in
# the FeatureView. In this case, airport_weather_fv uses DATETIME_UTC as its timestamp column, meaning each record in the
# spine will only be joined to weather prior to the current record.

query = """
    SELECT
        scheduled_departure_utc,
        airport_zip_code, -- join key for weather entity
        departure_code,
        destination_code,
        tickets_sold,
        plane_model, 
        tail_number -- join key for plane entity
    FROM us_flight_schedules
"""

spine_df = session.sql(query)

ds = fs.generate_dataset(
    name="trip_duration_ds",
    spine_df=spine_df,
    features=[plane_fv, airport_weather_fv],
    spine_timestamp_col="SCHEDULED_DEPARTURE_UTC",
    spine_label_cols=["TICKETS_SOLD"],
    include_feature_view_timestamp_col=False,   # optional
    version="V_001"
)

# Show preview of the Dataset contents by loading into a Pandas DataFrame
ds.read.to_pandas().head(5)

In [ ]:
session.sql("SHOW DATASETS").to_pandas()

# Conclusion

You can now use this dataset in your downstream modeling workloads. Models trained using Snowpark ML Modeling
and Snowflake Model Registry will automatically benefit from model lineage and other MLOps features.
You can find full examples of using the Snowflake Feature Store on GitHub at
[`snowflake-ml-python`](https://github.com/snowflakedb/snowflake-ml-python/tree/main/snowflake/ml/feature_store/notebooks/customer_demo>)